In [ ]:
#Bryan Bennett
# 3/2/2021

In [ ]:
from sagemaker import get_execution_role
import pandas as pd
import sagemaker
import boto3
import os

In [ ]:
role = get_execution_role()
sess = sagemaker.Session()

In [ ]:
bucketName = 'signet-ring-cell-bryan'
folderName = 'challenge2-colorectal-dataset/tissue-train-pos-v1/'

In [ ]:
s3_client = boto3.client('s3')
bucket = s3_client.list_objects(Bucket=bucketName, Prefix=folderName)

In [ ]:
len(bucket['Contents'])

In [ ]:
DST_folderName = 'challenge2-colorectal-dataset/tissue-train-pos-pngs/'

In [ ]:
#Create png copies of masks, someone check if this works with the file structure

#FIXME: Split train and validation sets here?
#import shutil
from PIL import Image


#Create annotation file for each image
for img_num in range(len(bucket['Contents'])):
    if img_num%2==0:
            file_name = bucket['Contents'][img_num]['Key']
            png_fileName = fileName.split('/')[-1][:-4] + '.png'
            with Image.open('{}/{}'.format(bucketName, file_name)) as im:
                s3_client.upload_fileobj(im, bucketName, DST_folderName + png_fileName)
                #im.save(bucketName + DST_folderName + file_name + '.png')

In [ ]:
# Bucket names for training and validation data

prefix = 'challenge2-colorectal-dataset/tumor_segmentation_model'

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'
train_annotation_channel = prefix + '/train_annotation'
validation_annotation_channel = prefix + '/validation_annotation'


s3_train_data = 's3://{}/{}'.format(bucketName, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucketName, validation_channel)
s3_train_annotation = 's3://{}/{}'.format(bucketName, train_annotation_channel)
s3_validation_annotation = 's3://{}/{}'.format(bucketName, validation_annotation_channel)

In [ ]:
# Split training and validation sets

ratio = 0.2 #Fraction of images to go in validation set (0.2 = 20%)

for img_num in range(len(bucket['Contents'])):
    
    file_name = bucket['Contents'][img_num]['Key']

    # it's a mask
    if img_num%2==0:    
        png_fileName = fileName.split('/')[-1][:-4] + '.png'
        with Image.open('{}/{}'.format(bucketName, file_name)) as im:     
            if(img_num < (img_num*ratio)):
                s3_client.upload_fileobj(im, bucketName, validation_annotation_channel + png_fileName)
            else:
                s3_client.upload_fileobj(im, bucketName, train_annotation_channel + png_fileName)
                
    # it's an image
    if img_num%2==1:
        with Image.open('{}/{}'.format(bucketName, file_name)) as im:     
            if(img_num < (img_num*ratio)):
                s3_client.upload_fileobj(im, bucketName, validation_annotation_channel + fileName)
            else:
                s3_client.upload_fileobj(im, bucketName, train_annotation_channel + fileName)
            

In [ ]:
import glob

num_training_samples = len(glob.glob1(s3_train_data,"*.jpg"))
num_validation_samples = len(glob.glob1(s3_validation_data,"*.jpg"))

print('Num Train Images = ' + str(num_training_samples))
assert num_training_samples == len(glob.glob1(s3_train_annotation,"*.png"))

print('Num Validation Images = ' + str(num_validation_samples))
assert num_validation_samples == len(glob.glob1(s3_validation_annotation,"*.png"))

In [ ]:
# Set output location where artifact will be stored
s3_output_location = 's3://{}/{}/output'.format(bucketName, 'challenge2-colorectal-dataset/tumor_segmentation_model')

In [ ]:
# Get training uri

#Arguments in-order
training_image = sagemaker.image_uris.retrieve(region=sess.boto_region_name, framework='semantic-segmentation')
print (training_image)

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'semantic-segmentation', repo_version="latest")
print (training_image)